# Obtain data of AGN from SIMBAD and filtering the objects with data of Ne II, Ne III and Ne V in infrared from NED 

In this notebook we develop the code to Obtain the data of AGNs type Sy1 (Seyfert 1), Sy2 (Seyfert 2) and SyG (Seyfert Galaxies) from **[SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-tap)** (SMB) and then search each object from the SMB sample in **[NED](https://ned.ipac.caltech.edu/)** to obtain the photometric data and filter all the objects from the sample that contain data of the emission lines of Ne II, Ne III and Ne V in infrared.

The part of the code to obtain the data from SIMBAD is based in Andres Ramos's work to obtain Seyfert Samples (https://github.com/aframosp/AGNView)

In [ ]:
# If this notebook will be run in Google Colab, first at all the next code must be run. If the notebook will be run in Jupyter
#then "comment" or omit this part 

import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    !pip install astroquery

In [1]:
from datetime import date
import time
import csv
import numpy as np

import pyvo
from astroquery.ned import Ned

import astropy.units as u
from astropy.table import hstack
from astropy.coordinates import SkyCoord

In [2]:
path='../Datos/Raw/'

Using the package **[PyVO](https://pyvo.readthedocs.io/en/latest/#)** we can access to remote data and services like the **TAP Service of SIMBAD [SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-tap)** to find and retrieve the astronomical data that we need, in this case: Seyfert Galaxies. To achive that, we use a script with **[ADQL](https://www.ivoa.net/documents/latest/ADQL.html) (Astronomical Data Query Language)**, which is based on SQL.

On the script we used the next **Field Names** for VOTable ouput:
- ra: Right ascension
- dec: Declination
- coo_bibcode: Bibliographical reference for coordinates
- otype_txt: Object type
- rvz_bibcode: Bibliographical reference of radial velocity and redshift
- rvz_radvel: Stored value. Either a radial velocity, or a redshift, depending on the rvz_type field 
- rvz_redshift
- rvz_type: Stored type of velocity: 'v'=radial velocity, 'z'=redshift
- nbref: bibliographical reference

~~~~sql
SELECT main_id
	,ra
	,dec
	,coo_bibcode
	,otype_txt
	,rvz_bibcode
	,rvz_radvel
	,rvz_redshift
	,rvz_type
	,nbref
	,alltypes.otypes
FROM basic
JOIN alltypes ON oid = oidref
WHERE basic.otype = 'Sy1'
	OR basic.otype = 'Sy2'
	OR basic.otype = 'SyG'
~~~~

In [3]:
service = pyvo.dal.TAPService("http://simbad.u-strasbg.fr:80/simbad/sim-tap")

result = service.search("""
SELECT main_id
    ,ra
    ,dec
    ,coo_bibcode
    ,otype_txt
    ,rvz_bibcode
    ,rvz_radvel
    ,rvz_redshift
    ,rvz_type
    ,nbref
    ,alltypes.otypes
FROM basic
JOIN alltypes ON oid = oidref
WHERE basic.otype = 'Sy1'
    OR basic.otype = 'Sy2'
    OR basic.otype = 'SyG'
""")

We save the date in a variable for use later in the file names of the results

In [4]:
today = date.today()
today

datetime.date(2021, 10, 15)

We extract the object name (or in this case the main id) from the sample result.

In [5]:
simbad_sample_Sy = result.to_table()
obj_name_SMB=simbad_sample_Sy["main_id"]
len(simbad_sample_Sy)

43708

The sample results is saved in a .csv file:

In [6]:
simbad_sample_Sy.write(path+'SMB_'+today.strftime('%b_%d_%Y')+'_Sy_Samples'+'.csv',
                    format='ascii.csv',overwrite=True)

Using **SkyCoord** from astropy we extract the coordinates information for each object from the SIMBAD samples. This is because for some objects when we search the information in [NED](https://ned.ipac.caltech.edu/) using their SIMBAD main id  it doesn't reconigze it, so we will use coordinates instead to get the NED object name

In [7]:
cat_smb_sample = SkyCoord(ra=simbad_sample_Sy['ra'], dec=simbad_sample_Sy['dec'])

The wavelenght for NeII, NeIII and NeV in the infrared spectrum are 12.8 microns, 15.6 microns and 14.3 microns respectively. This values are equivalents to the following frequencies which ara related to different labels of "Observed Passband" in NED database as we ilustrate in the next list:

* Frequency: 1.92e+13 Hz: [NeIII] 15.6 microns
* Frecuency: 1.93e+13 Hz: [Ne III] 15.56 (IRS), [NeIII] 15.6 Spitzer, [Ne III] (Spitzer), [Ne III] 15.5 (IRS)
* Frecuency: 2.09e+13 Hz: [Ne V] 14.32 (IRS), [Ne V] 14.32 Spitzer, [Ne V] 14.32, [Ne V] 14.3 Spitzer, [Ne V] 14.3 microns
* Frequency: 2.10e+13 Hz: [Ne V] 14.3 (IRS)
* Frequency: 2.34e+13 Hz: [Ne II] 12.81 (IRS), [Ne II] 12.8 Spitzer, [Ne II] 12.8 (IRS), [Ne II] 12.81, NEII (VLT), 12.81 microns (VLT) 

In the variable **Ne_IR_Fq** we define a list with the those frequency values

In [8]:
Ne_IR_Fq = [19200000000000.0,19300000000000.0,20900000000000.0,21000000000000.0,23400000000000.0]

The next part of the code use **[Astroquery](https://astroquery.readthedocs.io/en/latest/index.html)** package to get the "Frequency" column from the photometry table for each object of the SIMBAD sample or to get the NED object name from SIMBAD object coordiantes extracted before with SkyCoord. 

So, we will try to get the **"Photometry"** table using directly the SIMBAD object name. If this isn't recognized in NED, then we try to get the NED object name from the object coordinates and after that we will try again to get the "Photometry" table but now using the NED object name.

The filter for the objects with Ne Line Emissions in Infrared will be looking in the column **"Frequency"** of the table the values of the list defined in Ne_IR_Freq.

In [13]:
Ne_IR_obj = [] #List of objects with Ne-IR data

#We create and open a file .csv to write the list of objects names with Ne_IR data
outfile = open(path+'Obj_Ne-IR_'+today.strftime('%b_%d_%Y')+'.csv','w')
writer = csv.writer(outfile) # create the csv writer
header = ['Name_Id','id_from','RA','DEC','otype_txt','redshift','rvz_type','otypes','SMB_sample']#Creating header
writer.writerow(header) #Writing the header

print('Idx, Name ID, ID from\n')

start = time.time()
for ind, id_smb in enumerate(obj_name_SMB):
    try:
        phot_t = Ned.get_table(id_smb, table='photometry') #We get the Photometry Table using SIMBAD names
        Fr=phot_t['Frequency'] #We get the Frequency column from the table
        if Ne_IR_Fq[0] in Fr or Ne_IR_Fq[1] in Fr or Ne_IR_Fq[2] in Fr or Ne_IR_Fq[3] in Fr or Ne_IR_Fq[4] in Fr:
            print(f"{ind}, {id_smb}, SIMBAD")
            #Writing the object names
            outstring = [id_smb,'SIMBAD',simbad_sample_Sy["ra"][ind],simbad_sample_Sy["dec"][ind],
                         simbad_sample_Sy["otype_txt"][ind],simbad_sample_Sy["rvz_redshift"][ind],
                         simbad_sample_Sy["rvz_type"][ind],simbad_sample_Sy["otypes"][ind],ind]
            writer.writerow(outstring) #write the data
            Ne_IR_obj.append(id_smb) #Save the object name in the list Ne_IR_obj
    except:
        try:
            result_t = Ned.query_region(cat_smb_sample[ind],radius=0.0005 * u.deg) #For obtaining the data from NED for each sample 
            id_ned = result_t["Object Name"] 
            phot_t = Ned.get_table(id_ned, table='photometry')
            Fr=phot_t['Frequency']
            if Ne_IR_Fq[0] in Fr or Ne_IR_Fq[1] in Fr or Ne_IR_Fq[2] in Fr or Ne_IR_Fq[3] in Fr or Ne_IR_Fq[4] in Fr:
                print(f"{ind}, {id_ned[0]}, NED")
                #Writing the object names
                outstring = [id_ned[0],'NED',simbad_sample_Sy["ra"][ind],simbad_sample_Sy["dec"][ind],
                             simbad_sample_Sy["otype_txt"][ind],simbad_sample_Sy["rvz_redshift"][ind],
                             simbad_sample_Sy["rvz_type"][ind],simbad_sample_Sy["otypes"][ind],ind]
                writer.writerow(outstring) #write the data
                Ne_IR_obj.append(id_ned[0]) #Save the object name in the list Ne_IR_obj
        except:
            pass
end = time.time()

#we close the files with objects names
outfile.close()

print('\nTotal objects with lines emission of NeII or NeIII or NeV in IR  found: ',len(Ne_IR_obj))
print('Execution Time(seg): ' + str(end - start))
print('Execution Time(min): ' + str((end - start)/60))
print('Execution Time(hrs): ' + str((end - start)/3600))

Idx, Name ID, ID from

0, LEDA  165630, SIMBAD
23, ESO 362-18, SIMBAD
100, 3C 334, NED
182, ESO 323-77, SIMBAD
341, NGC  7479, SIMBAD
404, M  81, SIMBAD
412, NGC  1241, SIMBAD
413, NGC  3516, SIMBAD
433, IRAS 15206+3342, SIMBAD
454, LEDA 1487448, SIMBAD
615, Mrk  937, SIMBAD
628, ESO 297-18, SIMBAD
648, IC  450, SIMBAD
794, Mrk  622, SIMBAD
862, WISEA J035734.06-053719.6, NED
1055, 2MASS J21035875-0728025, SIMBAD
1115, IRAS 15480-0344, SIMBAD
1415, Mrk  471, SIMBAD
1589, 2MASS J00105486+0014510, SIMBAD
1592, WISEA J005439.78+155447.2, NED
1871, LEDA  166692, SIMBAD
2267, NGC  4992, SIMBAD
2624, 3C 295, SIMBAD
2851, LEDA    3237, SIMBAD
2852, Mrk  478, SIMBAD
2949, NGC  1125, SIMBAD
2966, Mrk  279, SIMBAD
2988, Mrk 1502, SIMBAD
2989, PB  3894, SIMBAD
2994, 3C 098, NED
3594, NGC  2992, SIMBAD
3664, NGC  2110, SIMBAD
3710, UGC  3351, SIMBAD
3768, NGC   788, SIMBAD
3772, NGC  4235, SIMBAD
3773, NGC  6860, SIMBAD
3844, Mrk  520, SIMBAD
3884, MCG+04-48-002, SIMBAD
4057, ICRF J145907.5+714019

34207, NGC  5347, SIMBAD
34227, MCG-03-58-007, SIMBAD
34260, 2MASX J12390630-1610472, SIMBAD
34300, IRAS 11215-2806, SIMBAD
34352, Mrk  883, SIMBAD
34560, IRAS F00183-7111, SIMBAD
34567, LEDA   37102, SIMBAD
34770, 2MASX J00215355-7910077, SIMBAD
34892, SDSS J120424.54+192509.7, SIMBAD
35629, 2MASS J10434845+5811477, SIMBAD
36058, 2XMM J142930.6+425148, SIMBAD
36225, SDSS J161647.32+371621.2, SIMBAD
36785, IRAS 03450+0055, SIMBAD
36861, WISEA J145331.51+135358.6, NED
36894, 3C 265.0, SIMBAD
37240, LEDA   50824, SIMBAD
37386, LEDA 2817285, SIMBAD
37389, NGC  6810, SIMBAD
37402, ICRF J090933.4+425346, SIMBAD
37419, ESO 209-12, SIMBAD
37553, LEDA   17155, SIMBAD
38016, ESO 506-27, SIMBAD
38204, LEDA  178130, SIMBAD
38436, WISEA J183835.40+355220.1, NED
38493, WISEA J134133.37-002432.1, NED
38614, 3C 343, SIMBAD
38628, LEDA 3095998, SIMBAD
38637, Mrk   22, SIMBAD
38638, Mrk 1066, SIMBAD
39136, 2XMM J135315.8+634546, SIMBAD
39137, 2E  2389, SIMBAD
41589, WISEA J010835.15+214818.5, NED
41735

Las celdas a continuación son para pruebas:

In [18]:
x=862

In [19]:
simbad_sample_Sy[x]

main_id,ra,dec,coo_bibcode,otype_txt,rvz_bibcode,rvz_radvel,rvz_redshift,rvz_type,nbref,otypes
,deg,deg,,,,km / s,,,,
object,float64,float64,object,object,object,float64,float64,str1,int32,object
[VV2003c] J035734.0-053719,59.39166666666667,-5.622138888888888,2007ApJS..172..634A,Sy1,2015ApJS..219...12A,54960.58310877802,0.20373,z,7,Sy1|AGN


In [17]:
outstring = [id_smb,'SIMBAD',simbad_sample_Sy["ra"][ind],simbad_sample_Sy["dec"][ind]]
outstring

['[VV2006c] J164424.0+255242', 'SIMBAD', 251.1001487379708, 25.8784356100449]

### Watermark

In [10]:
%load_ext watermark
%watermark -a "Jonhatan Bernal" -d -v -m
print('Specific Python packages')
%watermark -iv -w --packages astropy

Author: Jonhatan Bernal

Python implementation: CPython
Python version       : 3.8.8
IPython version      : 7.27.0

Compiler    : MSC v.1928 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : AMD64 Family 21 Model 2 Stepping 0, AuthenticAMD
CPU cores   : 8
Architecture: 64bit

Specific Python packages
astropy: 4.3.1

csv    : 1.0
astropy: 4.3.1
pyvo   : 1.1
numpy  : 1.20.3

Watermark: 2.2.0

